In [1]:
import torch
inputs = torch.tensor(
    [[
       [0.43, 0.15, 0.89], # Your     
       [0.55, 0.87, 0.66], # journey  
       [0.57, 0.85, 0.64], # starts   
       [0.22, 0.58, 0.33], # with     
       [0.77, 0.25, 0.10], # one      
       [0.05, 0.80, 0.55]  # step     
    ],
    [
       [0.43, 0.15, 0.89], # His     
       [0.55, 0.87, 0.66], # journey  
       [0.57, 0.85, 0.64], # end   
       [0.22, 0.58, 0.33], # at     
       [0.77, 0.25, 0.10], # last      
       [0.05, 0.80, 0.55]  # way     
    ]
])

In [2]:
inputs.shape

torch.Size([2, 6, 3])

In [3]:
import torch.nn as nn
class MaskedSelfAttention(nn.Module):
    def __init__(self,d_in,d_out, context_length, dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in,d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in,d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in,d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )
        
    def forward(self,x):
        no_of_input, no_of_token, embedding_dim = x.shape
        queries = self.W_query(x)
        keys = self.W_key(x)
        values = self.W_value(x)
        
        attention_scores = queries @ keys.transpose(1, 2)
        attention_scores = attention_scores.masked_fill(
            self.mask.bool()[:no_of_token , :no_of_token], -torch.inf
        )
        
        d_k = keys.shape[-1]
        attention_weights = torch.softmax(
            attention_scores / d_k**0.5 , dim=-1
        )
        attention_weights = self.dropout(attention_weights)
        
        context_vector = attention_weights @ values
        return context_vector
        

In [7]:
d_in = 3
d_out = 2
context_legth = inputs.shape[1]
print(d_in, d_out, context_legth)

3 2 6


In [8]:
maskedSA = MaskedSelfAttention(d_in,d_out,context_legth,0.5)

In [9]:
context_vector = maskedSA(inputs)

In [11]:
context_vector

tensor([[[ 0.0000,  0.0000],
         [-1.1032,  0.5015],
         [-0.4701,  0.2781],
         [-0.6667,  0.3971],
         [-0.4625,  0.2461],
         [-0.3613,  0.1834]],

        [[ 0.0000,  0.0000],
         [-0.7407,  0.4381],
         [-0.2316,  0.0405],
         [-0.3279,  0.1967],
         [-0.3195,  0.1279],
         [-0.4966,  0.2680]]], grad_fn=<UnsafeViewBackward0>)

In [12]:
context_vector.shape

torch.Size([2, 6, 2])